### The Data

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('../data/stars.csv')

In [12]:
df.sample(5)

,description,language,repo,stargazers,user
6163,An entity framework for Go,Go,facebookincubator/ent,1379,pushpendrapratap
11503,A library for adding scripting to .NET applica...,C#,microsoft/ClearScript,472,owen800q
19902,Headless chrome/chromium automation library (u...,Python,miyakogi/pyppeteer,2476,viseshrp
20416,In-App MongoDB Editor for Meteor (Meteor DevTo...,JavaScript,msavin/Mongol-meteor-explore-minimongo-devtools,844,kikeh
7485,Python Rest Testing,Python,svanoort/pyresttest,935,victordomingos


### The Cleanup

In [14]:
df = df[df['repo'] != 'maxhumber/gazpacho']
df = df[df.language.isin(['Python', 'Jupyter Notebook'])]
popular = pd.DataFrame(df['repo'].value_counts())
select_repos = popular[popular['repo'] >= 5].index.tolist()
df = df[df['repo'].isin(select_repos)]

In [15]:
df.sample(5)

,description,language,repo,stargazers,user
3065,"Out-of-Core DataFrames for Python, visualize a...",Python,vaexio/vaex,1684,Na0ture
3009,GluonTS - Probabilistic Time Series Modeling i...,Python,awslabs/gluon-ts,667,Na0ture
12247,The next generation relational database.,Python,edgedb/edgedb,3293,jamiehall007
9884,Project documentation with Markdown.,Python,mkdocs/mkdocs,8436,erhan
22612,Library to scrape and clean web pages to creat...,Python,chiphuyen/lazynlp,1567,andrewwxy


In [16]:
df.shape

(2229, 5)

In [9]:
# shove into something that looks like this?
pd.DataFrame([
    [0, 0, 1, 0],
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 0, 0, 1]
])

,0,1,2,3
0,0,0,1,0
1,0,1,1,0
2,1,0,0,1
3,0,0,0,1


### The Preparation

In [17]:
df = df.groupby(['user'])['repo'].apply(lambda x: ','.join(x))
df = pd.DataFrame(df)

In [18]:
df.sample(5)

,repo
user,
lsantosdemoura,"laike9m/pdir2,hugapi/hug,timothycrosley/portra..."
fahmihidayah,"hack4impact/flask-base,faif/python-patterns,py..."
mxuribe,"sebst/pythonic-news,facebookresearch/hydra,koo..."
Brombult,"mahmoud/awesome-python-applications,codelucas/..."
periode,"deepfakes/faceswap,minimaxir/big-list-of-naugh..."


In [19]:
df.loc['Curlybear'].values.tolist()

['vinta/awesome-python,CorentinJ/Real-Time-Voice-Cloning,danijar/handout,ageitgey/face_recognition,jofpin/trape,grapheneX/grapheneX,gto76/python-cheatsheet,trekhleb/homemade-machine-learning,manrajgrover/halo,chris104957/maildown,psf/black,stewartmcgown/uds,TheAlgorithms/Python,cool-RR/PySnooper,Bogdanp/molten,nicolargo/glances,mkaz/termgraph,donnemartin/system-design-primer,chubin/cheat.sh,psf/requests-html,Zulko/moviepy,jarun/Buku,pypa/pipenv,soimort/you-get,public-apis/public-apis,HelloZeroNet/ZeroNet']

### The Model

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors


class NNRecommender:
    def __init__(self, n_neighbors=10, max_features=1000, tokenizer=lambda x: x.split(",")):
        self.cv = CountVectorizer(tokenizer=tokenizer, max_features=max_features)
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)

    def fit(self, X):
        self.X = X
        X = self.cv.fit_transform(X)
        self.nn.fit(X)
        return self

    def predict(self, X):
        Xp = []
        for Xi in X:
            Xt = self.cv.transform([Xi])
            neighbors = self.nn.kneighbors(Xt, return_distance=False)
            repos = []
            for n in neighbors[0]:
                r = self.X.iloc[int(n)].split(",")
                repos.extend(r)
            repos = list(set(repos))
            repos = [r for r in repos if r not in Xi.split(",")]
            Xp.append(repos)
        return Xp

In [21]:
model = NNRecommender()
model.fit(df['repo'])

In [22]:
df['repo'][42]

'CorentinJ/Real-Time-Voice-Cloning,s0md3v/goop,psf/requests,slundberg/shap,facebookresearch/pytext,practicalAI/practicalAI,Zulko/moviepy,python/mypy,ParthS007/background,keon/algorithms,google/python-fire,eriklindernoren/ML-From-Scratch,minimaxir/big-list-of-naughty-strings,pudo/dataset,nvbn/thefuck,dbcli/pgcli,faif/python-patterns,chriskiehl/Gooey'

In [23]:
model.predict([df['repo'][42]])

[['smacke/subsync',
  'ageron/handson-ml2',
  'weskerfoot/DeleteFB',
  'sherlock-project/sherlock',
  'Miserlou/Zappa',
  'shengqiangzhang/examples-of-web-crawlers',
  'sloria/TextBlob',
  'deepfakes/faceswap',
  'fastai/fastai',
  'dae/anki',
  'scikit-learn/scikit-learn',
  'donnemartin/interactive-coding-challenges']]

### Under the Hood

In [24]:
df.iloc[0].values.tolist()

['python/cpython,hugapi/hug,HypothesisWorks/hypothesis,timothycrosley/portray,timothycrosley/hypothesis-auto,tiangolo/fastapi,bocadilloproject/bocadillo,tartiflette/tartiflette,encode/httpx,pytest-dev/pytest,tonybaloney/wily,sdispater/poetry,mahmoud/glom,psf/black,chubin/wttr.in']

In [25]:
cv = CountVectorizer(tokenizer=lambda x: x.split(','), max_features=1000)
X = cv.fit_transform(df['repo'])
X.todense()[:5]

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [26]:
nn = NearestNeighbors(n_neighbors=10)

In [27]:
nn.fit(X)
ind = nn.kneighbors(return_distance=False)
ind[:5]

array([[106,  14,  99,  48,  65,   3,  10,  19,   2,  40],
       [ 14,  66,  99,  10,  19,  65,  48,   3,  40,  12],
       [ 10,  19,   3,  99,  48,  14,  65,  40,  66,  92],
       [ 48,  10,  99,  19,  14,  65,   2,  40,  92,  72],
       [ 28,  10,  48,  72,  20,  19,  99,  14,  65,   3]])

In [28]:
df.iloc[106].values

array(['timothycrosley/hypothesis-auto,timothycrosley/portray,sdispater/poetry,wyounas/homer,tiangolo/fastapi,psf/black,marshmallow-code/marshmallow,falconry/falcon,cookiecutter/cookiecutter'],
      dtype=object)

#### Pickle 

In [29]:
import pickle

In [30]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

PicklingError: Can't pickle <function NNRecommender.<lambda> at 0x122b7dd08>: attribute lookup NNRecommender.<lambda> on __main__ failed

In [31]:
import dill # pip install dill

with open('model.pkl', 'wb') as f:
    dill.dump(model, f)

del model

with open('model.pkl', 'rb') as f:
    model = dill.load(f)

In [32]:
model.predict(['streamlit/streamlit,huggingface/transformers,encode/httpx,aws/chalice,maxhumber/chart'])

[['sloria/TextBlob',
  'donnemartin/system-design-primer',
  'shengqiangzhang/examples-of-web-crawlers',
  'vinta/awesome-python',
  'plasticityai/supersqlite',
  'fastai/fastai',
  'scikit-learn/scikit-learn',
  'TheAlgorithms/Python',
  'mherrmann/fbs',
  'ytdl-org/youtube-dl',
  'PySimpleGUI/PySimpleGUI',
  'minimaxir/big-list-of-naughty-strings',
  'Avik-Jain/100-Days-Of-ML-Code']]